In [1]:
# Cell 1: Install All Required Libraries
!pip install -U langchain langchain-community langchain-ollama faiss-cpu

print("--- All libraries installed. ---")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.5 MB/s eta 0:00:00
--- All libraries installed. ---


In [2]:
# Cell 2: Download and Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

print("--- Ollama installation script finished. ---")

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
--- Ollama installation script finished. ---


In [5]:
# Cell 3: Pull the Llama 3 Model
!ollama pull llama3

In [8]:
# Cell 4 (Corrected Version to fix "address in use" error)

# First, kill any old Ollama processes that might be running.
print("--- Stopping any old Ollama server processes... ---")
!pkill ollama
import time
time.sleep(3) # Give it a moment to fully shut down.

# Now, start a fresh server and log its output.
print("--- Starting a new, clean Ollama server... ---")
!nohup ollama serve > ollama.log 2>&1 &
time.sleep(10) # Give it 10 seconds to initialize.

print("--- Server started. Now checking the log... ---")

--- Stopping any old Ollama server processes... ---
--- Starting a new, clean Ollama server... ---
--- Server started. Now checking the log... ---


In [9]:
# Cell 5: VERIFY GPU is Active
# This command prints the contents of the log file.
!cat ollama.log

time=2025-08-14T07:37:19.139Z level=INFO source=routes.go:1304 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false ROCR_VISIBLE_DEVICE

In [11]:
# Cell 6: Load "Alice in Wonderland" Text File
from langchain_community.document_loaders import TextLoader

try:
    documents = TextLoader("alice_in_wonderland.txt").load()
    print(f"Successfully loaded {len(documents[0].page_content)} characters from the book.")
except Exception as e:
    print(f"Error loading file: {e}")
    print("\nPLEASE UPLOAD 'alice_in_wonderland.txt' TO THE COLAB FILES.")

Successfully loaded 148574 characters from the book.


In [12]:
# Cell 7: Build the RAG Pipeline (Optimized Version)
import time
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings  # Use the new recommended import
from langchain_ollama import ChatOllama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

print("--- Starting RAG Pipeline ---")

# 1. Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

# 2. Create the Ollama embeddings model
embeddings = OllamaEmbeddings(model="llama3")

# 3. Create embeddings for all chunks in a single batch call
print(f"Creating embeddings for {len(splits)} chunks... (This should be fast on a GPU)")
start_time = time.time()
all_texts = [s.page_content for s in splits]
all_embeddings = embeddings.embed_documents(all_texts)
end_time = time.time()
print(f"Embeddings created successfully in {end_time - start_time:.2f} seconds.")

# 4. Create the FAISS vector store from the pre-computed embeddings
print("Creating FAISS vector store...")
vectorstore = FAISS.from_texts(texts=all_texts, embedding=embeddings, metadatas=[s.metadata for s in splits])
print("Vector store created successfully.")

# 5. Create the rest of the RAG chain
llm = ChatOllama(model="llama3")
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context. Provide a detailed answer from the book. If the answer is not in the context, say "I cannot find that information in the book."

<context>
{context}
</context>

Question: {input}""")
document_chain = create_stuff_documents_chain(llm, prompt)
retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

print("\n--- RAG Chain is ready! You can now ask questions. ---")

--- Starting RAG Pipeline ---
Creating embeddings for 195 chunks... (This should be fast on a GPU)
Embeddings created successfully in 49.23 seconds.
Creating FAISS vector store...
Vector store created successfully.

--- RAG Chain is ready! You can now ask questions. ---


In [14]:
# Cell 8: Ask Your First Question!
question = "Who is Alice?"

print(f"Question: {question}\n")
print("--- Thinking... ---")

response = retrieval_chain.invoke({"input": question})

print("\n--- Answer ---")
print(response["answer"])

Question: Who is Alice?

--- Thinking... ---

--- Answer ---
Based on the provided context, Alice refers to the main character in the story. The passage describes her thoughts, actions, and emotions throughout the scene. She is the one who has forgotten the little golden key and tries to reach it but fails, eventually sitting down and crying.
